In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("Manhattan12.csv", skiprows=4)
df.shape

(27395, 21)

In [3]:
df.columns = df.columns.str.replace("\n", "").str.replace(" ", "_")
df.rename(columns={"SALEPRICE": "SALE_PRICE", "APARTMENTNUMBER": "APARTMENT_NUMBER"}, inplace=True)

In [4]:
df.columns = df.columns.str.strip().str.replace("\n", "")

def format_and_convert_column_to_numeric(col, dtype):
    df[col] = df[col].str.replace(",","").str.replace("$","").astype(dtype)
    
format_and_convert_column_to_numeric('SALE_PRICE', np.int64)
format_and_convert_column_to_numeric('GROSS_SQUARE_FEET', np.float64)
format_and_convert_column_to_numeric('LAND_SQUARE_FEET', np.float64)
format_and_convert_column_to_numeric('RESIDENTIAL_UNITS', np.int64)
format_and_convert_column_to_numeric('TOTAL_UNITS', np.int64)

# Convert To datetime
df['SALE_DATE'] =pd.to_datetime(df['SALE_DATE'], dayfirst=True)

df['YEAR_BUILT'] = df['YEAR_BUILT'].astype(np.int64)

/var/folders/4m/__j4737x1p73lg1hgfht7tnm0000gn/T/ipykernel_78151/1126458197.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace(",","").str.replace("$","").astype(dtype)


In [5]:
categorical = df.select_dtypes(include='object').columns.tolist()
numerical = df.select_dtypes(include=['int', 'float']).columns.tolist()
print("Categorical columns: ", categorical)
print("Numerical columns: ", numerical)

Categorical columns:  ['NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT', 'EASE-MENT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'BUILDING_CLASS_AT_TIME_OF_SALE']
Numerical columns:  ['BOROUGH', 'BLOCK', 'LOT', 'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE', 'SALE_PRICE']


In [6]:
df[categorical] = df[categorical].applymap(lambda x: x.replace(r" +", ""))
df[categorical] = df[categorical].applymap(lambda x: " ".join(x.split()))

In [7]:
df[categorical] = df[categorical].replace("", np.nan)
df[numerical] = df[numerical].replace(0, np.nan)

In [8]:
df.isna().sum() / df.shape[0] * 100

BOROUGH                             0.000000
NEIGHBORHOOD                        0.062055
BUILDING_CLASS_CATEGORY            12.608140
TAX_CLASS_AT_PRESENT                0.521993
BLOCK                               0.000000
LOT                                 0.000000
EASE-MENT                         100.000000
BUILDING_CLASS_AT_PRESENT           0.521993
ADDRESS                             0.000000
APARTMENT_NUMBER                   53.184888
ZIP_CODE                            0.003650
RESIDENTIAL_UNITS                  59.762730
COMMERCIAL_UNITS                   87.468516
TOTAL_UNITS                        35.422522
LAND_SQUARE_FEET                   83.613798
GROSS_SQUARE_FEET                  84.208797
YEAR_BUILT                         23.511590
TAX_CLASS_AT_TIME_OF_SALE           0.000000
BUILDING_CLASS_AT_TIME_OF_SALE      0.000000
SALE_PRICE                         27.716737
SALE_DATE                           0.000000
dtype: float64

In [9]:
df.drop(columns=['BOROUGH', 'EASE-MENT', 'APARTMENT_NUMBER'], inplace=True)
df.columns

Index(['NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT',
       'BLOCK', 'LOT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [10]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.isna().sum()

NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64